<a href="https://colab.research.google.com/github/Yeaminul/Applied-Machine-Learning-in-Python/blob/master/Handwritten_Digit_Recogniser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive//'My Drive/Numta'

/content/drive/My Drive/Numta


In [5]:
ls

code/  data/  Final_DB/  model/


In [0]:
#Loading the data set and variable assignment
train = pd.read_csv("data/digit-recognizer/train.csv")
test = pd.read_csv("data/digit-recognizer/test.csv")
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 
Y_train = to_categorical(Y_train, num_classes = 10) #One hot encoding

In [8]:
#Checking for NULL values
X_train.isnull().any().describe()
test.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [0]:
# As pixel values are within 0-255, it's a good normalization technique to scale them from 0-1
X_train = X_train / 255
test = test / 255

In [0]:
# perfroming dimensional tweakings for the network inputs and spliting the data set according to 90:10 ratio
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [12]:
#Architecture
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [14]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
epochs = 30
batch_size = 64
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [15]:
final = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val),callbacks=[learning_rate_reduction])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 37800 samples, validate on 4200 samples
Epoch 1/30





37800/37800 [==============================] - 262s 7ms/step - loss: 0.2449 - acc: 0.9238 - val_loss: 0.0770 - val_acc: 0.9790
Epoch 2/30
37800/37800 [==============================] - 257s 7ms/step - loss: 0.1000 - acc: 0.9724 - val_loss: 0.0579 - val_acc: 0.9845
Epoch 3/30
37800/37800 [==============================] - 256s 7ms/step - loss: 0.0884 - acc: 0.9768 - val_loss: 0.0483 - val_acc: 0.9862
Epoch 4/30
37800/37800 [==============================] - 260s 7ms/step - loss: 0.0792 - acc: 0.9795 - val_loss: 0.0557 - val_acc: 0.9829
Epoch 5/30
37800/37800 [==============================] - 257s 7ms/step - loss: 0.0797 - acc: 0.9808 - val_loss: 0.0502 - val_acc: 0.9874
Epoch 6/30
37800/37800 [==============================] - 256s 7ms/step - loss: 0.0804 - acc: 0.9818 - val_loss: 0.0557 - val_acc: 0.9883
Epoch 7/30
37800/3780

In [0]:
model.save("trained_model.h5")

In [0]:
results = model.predict(test)